In [54]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf
# import knn regression
from sklearn.neighbors import KNeighborsRegressor
# test train split
from sklearn.model_selection import train_test_split
import warnings

In [55]:
warnings.filterwarnings('ignore')

In [56]:
df = yf.download('RELIANCE.NS', start='2014-01-01', end='2023-08-07', progress=False)
df = pd.DataFrame(df)
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-07-31,2527.850098,2553.899902,2517.949951,2549.250000,2549.250000,4275587
2023-08-01,2555.000000,2559.000000,2505.300049,2513.199951,2513.199951,4768316
2023-08-02,2509.199951,2512.000000,2463.600098,2486.350098,2486.350098,6374107
2023-08-03,2475.300049,2500.899902,2458.050049,2475.899902,2475.899902,5084175
2023-08-04,2498.800049,2516.000000,2471.600098,2509.550049,2509.550049,11442805


In [57]:
df = df.reset_index('Date')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-01-01,444.511475,444.907715,439.632721,440.227081,414.639160,2699496
1,2014-01-02,439.682251,443.793274,430.964874,433.391876,408.201263,2779905
2,2014-01-03,431.707855,432.574615,423.485779,428.067352,403.186279,5922841
3,2014-01-06,427.002441,427.572052,421.009247,423.460999,398.847595,6145266
4,2014-01-07,423.287659,425.937531,416.080963,417.096344,392.852905,7989240


In [58]:
df["Date"]= pd.to_datetime(df["Date"])
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-01-01,444.511475,444.907715,439.632721,440.227081,414.639160,2699496
1,2014-01-02,439.682251,443.793274,430.964874,433.391876,408.201263,2779905
2,2014-01-03,431.707855,432.574615,423.485779,428.067352,403.186279,5922841
3,2014-01-06,427.002441,427.572052,421.009247,423.460999,398.847595,6145266
4,2014-01-07,423.287659,425.937531,416.080963,417.096344,392.852905,7989240


In [59]:
df.sort_values(by='Date', inplace=True)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-01-01,444.511475,444.907715,439.632721,440.227081,414.639160,2699496
1,2014-01-02,439.682251,443.793274,430.964874,433.391876,408.201263,2779905
2,2014-01-03,431.707855,432.574615,423.485779,428.067352,403.186279,5922841
3,2014-01-06,427.002441,427.572052,421.009247,423.460999,398.847595,6145266
4,2014-01-07,423.287659,425.937531,416.080963,417.096344,392.852905,7989240


In [60]:
df["Prev_Close"] = df["Close"].shift(1)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Prev_Close
0,2014-01-01,444.511475,444.907715,439.632721,440.227081,414.639160,2699496,NaN
1,2014-01-02,439.682251,443.793274,430.964874,433.391876,408.201263,2779905,440.227081
2,2014-01-03,431.707855,432.574615,423.485779,428.067352,403.186279,5922841,433.391876
3,2014-01-06,427.002441,427.572052,421.009247,423.460999,398.847595,6145266,428.067352
4,2014-01-07,423.287659,425.937531,416.080963,417.096344,392.852905,7989240,423.460999


In [61]:
df["Prev_High"] = df["High"].shift(1)
df["Prev_Low"] = df["Low"].shift(1)
df["Prev_Open"] = df["Open"].shift(1)


In [62]:
df.dropna(inplace=True)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Prev_Close,Prev_High,Prev_Low,Prev_Open
1,2014-01-02,439.682251,443.793274,430.964874,433.391876,408.201263,2779905,440.227081,444.907715,439.632721,444.511475
2,2014-01-03,431.707855,432.574615,423.485779,428.067352,403.186279,5922841,433.391876,443.793274,430.964874,439.682251
3,2014-01-06,427.002441,427.572052,421.009247,423.460999,398.847595,6145266,428.067352,432.574615,423.485779,431.707855
4,2014-01-07,423.287659,425.937531,416.080963,417.096344,392.852905,7989240,423.460999,427.572052,421.009247,427.002441
5,2014-01-08,418.235535,424.649750,418.235535,420.538727,396.095184,5873245,417.096344,425.937531,416.080963,423.287659


In [63]:
df["Prev_CO_Diff"] = df["Prev_Close"] - df["Prev_Open"]
df["Prev_HL_Diff"] = df["Prev_High"] - df["Prev_Low"]

df.head()
df.drop(["Date"],axis=1,inplace=True)
df.head()
# add new column time 
df["Time"] = df.index 


In [64]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,Prev_Close,Prev_High,Prev_Low,Prev_Open,Prev_CO_Diff,Prev_HL_Diff,Time
1,439.682251,443.793274,430.964874,433.391876,408.201263,2779905,440.227081,444.907715,439.632721,444.511475,-4.284393,5.274994,1
2,431.707855,432.574615,423.485779,428.067352,403.186279,5922841,433.391876,443.793274,430.964874,439.682251,-6.290375,12.828400,2
3,427.002441,427.572052,421.009247,423.460999,398.847595,6145266,428.067352,432.574615,423.485779,431.707855,-3.640503,9.088837,3
4,423.287659,425.937531,416.080963,417.096344,392.852905,7989240,423.460999,427.572052,421.009247,427.002441,-3.541443,6.562805,4
5,418.235535,424.649750,418.235535,420.538727,396.095184,5873245,417.096344,425.937531,416.080963,423.287659,-6.191315,9.856567,5


In [65]:

df.tail()

,Open,High,Low,Close,Adj Close,Volume,Prev_Close,Prev_High,Prev_Low,Prev_Open,Prev_CO_Diff,Prev_HL_Diff,Time
2362,2527.850098,2553.899902,2517.949951,2549.250000,2549.250000,4275587,2527.850098,2542.850098,2500.550049,2512.050049,15.800049,42.300049,2362
2363,2555.000000,2559.000000,2505.300049,2513.199951,2513.199951,4768316,2549.250000,2553.899902,2517.949951,2527.850098,21.399902,35.949951,2363
2364,2509.199951,2512.000000,2463.600098,2486.350098,2486.350098,6374107,2513.199951,2559.000000,2505.300049,2555.000000,-41.800049,53.699951,2364
2365,2475.300049,2500.899902,2458.050049,2475.899902,2475.899902,5084175,2486.350098,2512.000000,2463.600098,2509.199951,-22.849854,48.399902,2365
2366,2498.800049,2516.000000,2471.600098,2509.550049,2509.550049,11442805,2475.899902,2500.899902,2458.050049,2475.300049,0.599854,42.849854,2366


In [66]:
def train(df):
    X = df.drop(['Open','High','Low','Close','Volume','Adj Close'], axis=1)
    y_close = df['Close']
    y_open = df['Open']
    y_high = df['High']
    y_low = df['Low']

    # KNN
    knn_close = KNeighborsRegressor(n_neighbors=6)
    knn_close.fit(X, y_close)
    print(r2_score(y_close, knn_close.predict(X)))
    knn_open = KNeighborsRegressor(n_neighbors=6)
    knn_open.fit(X, y_open)
    knn_high = KNeighborsRegressor(n_neighbors=6)
    knn_high.fit(X, y_high)
    knn_low = KNeighborsRegressor(n_neighbors=6)
    knn_low.fit(X, y_low)

    return knn_close, knn_open, knn_high, knn_low



In [67]:
def getLast(df):
    openn = df['Open'].iloc[-1]
    close = df['Close'].iloc[-1]
    high = df['High'].iloc[-1]
    low = df['Low'].iloc[-1]
    co_diff = close - openn
    hl_diff = high - low
    time = df['Time'].iloc[-1]
    x = [[close,high,low,openn,co_diff,hl_diff,time]]
    return x

In [68]:
def appendLast(openn,close,high,low,df):
    prev_open = df['Open'].iloc[-1]
    prev_close = df['Close'].iloc[-1]
    prev_high = df['High'].iloc[-1]
    prev_low = df['Low'].iloc[-1]
    prev_co_diff = prev_close - prev_open
    prev_hl_diff = prev_high - prev_low
    new_row = {'Open':openn,'Close':close,'High':high,'Low':low,'Prev_Open':prev_open,'Prev_Close':prev_close,'Prev_High':prev_high,'Prev_Low':prev_low,'Prev_CO_Diff':prev_co_diff,'Prev_HL_Diff':prev_hl_diff,'Time':df['Time'].iloc[-1]+1}
    df = df.append(new_row,ignore_index=True)
    return df

In [69]:
def predictNextFifteen(df):
    for i in range(15):
        knn_close, knn_open, knn_high, knn_low = train(df)
        x = getLast(df)
        close = knn_close.predict(x)[0]
        openn = knn_open.predict(x)[0]
        high = knn_high.predict(x)[0]
        low = knn_low.predict(x)[0]
        df = appendLast(openn,close,high,low,df)
    return df

        


In [70]:
df = predictNextFifteen(df)
df.tail(15)

0.9990316356643738
0.9990325023747585
0.9990328703443819
0.9990338909136359
0.9990346564653318
0.999036187161384
0.9990372962769863
0.9990397341112415
0.9990407073474672
0.9990416612168967
0.9990426210062043
0.9990435912176721
0.9990445439559127
0.9990454952000682
0.9990464452675574


,Open,High,Low,Close,Adj Close,Volume,Prev_Close,Prev_High,Prev_Low,Prev_Open,Prev_CO_Diff,Prev_HL_Diff,Time
2366,2494.716675,2519.008301,2487.191650,2505.441650,NaN,NaN,2509.550049,2516.000000,2471.600098,2498.800049,10.750000,44.399902,2367.0
2367,2499.094462,2521.843018,2492.206909,2509.931925,NaN,NaN,2505.441650,2519.008301,2487.191650,2494.716675,10.724976,31.816650,2368.0
2368,2499.943539,2524.616862,2494.241394,2513.953905,NaN,NaN,2509.931925,2521.843018,2492.206909,2499.094462,10.837463,29.636108,2369.0
2369,2506.509121,2528.469672,2500.839976,2521.146255,NaN,NaN,2513.953905,2524.616862,2494.241394,2499.943539,14.010366,30.375468,2370.0
2370,2508.185649,2533.806292,2501.571647,2525.262306,NaN,NaN,2521.146255,2528.469672,2500.839976,2506.509121,14.637134,27.629696,2371.0
2371,2516.097145,2536.939291,2502.018321,2522.124057,NaN,NaN,2525.262306,2533.806292,2501.571647,2508.185649,17.076656,32.234646,2372.0
2372,2509.613336,2533.262506,2501.471366,2523.611408,NaN,NaN,2522.124057,2536.939291,2502.018321,2516.097145,6.026912,34.920970,2373.0
2373,2511.366481,2535.165754,2503.015443,2525.891322,NaN,NaN,2523.611408,2533.262506,2501.471366,2509.613336,13.998072,31.791140,2374.0
2374,2513.270305,2536.923903,2504.477784,2527.880891,NaN,NaN,2525.891322,2535.165754,2503.015443,2511.366481,14.524841,32.150312,2375.0
2375,2514.397169,2538.332942,2505.084085,2529.003331,NaN,NaN,2527.880891,2536.923903,2504.477784,2513.270305,14.610586,32.446119,2376.0
